<a href="https://colab.research.google.com/github/voke-brume/AI-ML/blob/main/AI/ComputerVision/ConvolutionCornersTransformation/ConvCornTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Image Filtering**

## **Convolution**

In [ ]:
# TODO: Complete the function
def convolve(image, kernel):
  """
  Return the convolution result: image * kernel.
  Reminder to implement convolution and not cross-correlation!
  Caution: Please use zero-padding.

  Input- image: H x W
          kernel: h x w
  Output- convolve: H x W
  """
  # Reshape image if image is not 2D
  if len(image.shape) > 2: image = reshape_image(image)

  # Pad image by appropriate pixels
  padded_image = zero_padding(image, get_padding_on_sides(kernel.shape[0]))

  # Flip kernel for convolution
  if kernel.shape[0] == 1: flipped_kernel = np.fliplr( kernel )
  elif kernel.shape[1] == 1: flipped_kernel = np.flipud( kernel )
  else: flipped_kernel = np.flipud(np.fliplr(kernel))

  # Create output with same size as image but filled with zeros
  output = np.zeros(shape = image.shape)

  # Save kernel size
  k = max( flipped_kernel.shape[0], flipped_kernel.shape[1] )

  # Iterate over the rows
  for i in range(image.shape[0]):
      # Iterate over the columns
      for j in range(image.shape[1]):
          # Get the current matrix
          matrix = padded_image[i:i+k, j:j+k]
          # Apply convolution using element-wise multiplication and summation
          output[i, j] = np.sum( np.multiply(matrix, flipped_kernel) )

  return output

## **Gaussian Kernel**

In [ ]:
# Gaussian Filter Implementation
def gaussian_kernel(size: int, sigma: float, sum_of_one = False) -> np.array:
  """
  This function creates a gaussian filter of specified size and
  standard deviation (sigma).

  Input - size(int)
          sigma (float)
          sum_of_one (boolean): determines if the sum of kernel values is one or not
  """
  ax = np.linspace(-(size - 1) / 2., (size - 1) / 2., size)
  xx, yy = np.meshgrid(ax, ax)

  kernel = np.exp(-0.5 * (np.square(xx) + np.square(yy)) / np.square(sigma))

  # Return sum of one
  if sum_of_one == True:  return kernel / np.sum(kernel)

  # Return values that do not sum up to one
  return kernel

## **Edge Detection**

In [ ]:
# TODO: Complete the function
def edge_detection(image):
    """
    Return Ix, Iy and the gradient magnitude of the input image

    Input- image: H x W
    Output- Ix, Iy, grad_magnitude: H x W
    """
    
    # TODO: Fix kx, ky
    kx = np.array([ [-1, 0, 1] ])  # 1 x 3
    ky = np.array([ [1], [0], [-1] ])  # 3 x 1

    #  Convolve image and derivative filters
    Ix = convolve(image, kx)
    Iy = convolve(image, ky)

    # TODO: Use Ix, Iy to calculate grad_magnitude: (Ix^2 + Iy^2)^1/2
    grad_magnitude = np.sqrt( np.square(Ix) + np.square(Iy) )

    return Ix, Iy, grad_magnitude

# **Corners**

## **Corner Score**

In [ ]:
# TODO: Complete the function
def corner_score(image, u=5, v=5, window_size=(5, 5)):
  """
  Given an input image, x_offset, y_offset, and window_size,
  return the function E(u,v) for window size W
  corner detector score for that pixel.
  Use zero-padding to handle window values outside of the image.

  Input- image: H x W
          u: a scalar for x offset
          v: a scalar for y offset
          window_size: a tuple for window size

  Output- results: a image of size H x W
  """
  # Save window size
  wind_size = window_size[0]

  # Reshape image if image is not 2D
  if len(image.shape) > 2: image = reshape_image(image)

  # Save image shape
  image_shape = image.shape

  # Pad image by appropriate pixels
  padded_image = zero_padding(image, get_padding_on_sides(wind_size) +  wind_size)
  #padded_image = padded_image[wind_size:, wind_size:]

  # Create output with same size as image but filled with zeros
  output = np.zeros(shape = image_shape)

   # Iterate over the rows
  for i in range(image_shape[0]):
      # Iterate over the columns
      for j in range(image_shape[1]):
          # img[i, j] = individual pixel value
          # Get the current matrix
          current_matrix = padded_image[i+wind_size: i+ (wind_size*2), j+wind_size:j+ (wind_size*2)]
          rolled_matrix = np.roll(np.roll(padded_image, padded_image.shape[1] - u, axis=1), padded_image.shape[0] - v, axis=0)[i+wind_size: i+ (wind_size*2), j+wind_size:j+ (wind_size*2)]

          # Calculate for E(u,v) for each pixel
          output[i, j] = np.sum( np.square(rolled_matrix - current_matrix) )

  # output = None
  return output

## **Harris Corner Detector**

In [ ]:
# TODO: Complete the function
def harris_detector(image, window_size=(5, 5)):
    """
    Given an input image, calculate the Harris Detector score for all pixels
    You can use same-padding for intensity (or 0-padding for derivatives)
    to handle window values outside of the image.

    Input- image: H x W
    Output- results: a image of size H x W
    """

    # Change image to grayscale
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # compute the derivatives
    Ix, Iy, mag = edge_detection(image)

    # Gaussian kernel
    gauss_kernel = gaussian_kernel(window_size[0], 1, sum_of_one = True)

    Ixx = ndimage.convolve(np.square(Ix), gauss_kernel, mode='constant', cval=0.0)
    Iyy = ndimage.convolve(np.square(Iy), gauss_kernel, mode='constant', cval=0.0)
    Ixy = ndimage.convolve(Ix * Iy, gauss_kernel, mode='constant', cval=0.0)

    # For each image location, construct the structure tensor and calculate
    # the Harris response
    sensitivity_factor = 0.05

    # Parameters to computer Harri response
    detA = Ixx * Iyy - np.square(Ixy)
    traceA = Ixx +  Iyy
    
    # Response scores for harri detector
    response = detA - sensitivity_factor * np.square(traceA)

    # Normalizing the values
    response_range = response.max() - response.min()
    response = (response / response_range) * 255

    return response

# **Fitting Affine Transform**

In [ ]:
# Function for fitting affine transform using least squared method
def affine_transform_lstsq(file):
  # Seperate the values
  x_y = file[:,0:2]
  x_y_prime = file[:,2:4]
  plt.scatter(x_y[:,0],x_y[:,1],1)
  plt.scatter(x_y_prime[:,0],x_y_prime[:,1],1)
  # Create empty matrix of values
  x_y_prime = [item for sublist in zip(x_y_prime[:,0], x_y_prime[:,1]) for item in sublist]

  z=np.zeros_like(x_y)
  a1 = np.array([item for sublist in zip(x_y[:,0:2],z[:,0:2]) for item in sublist])
  a2 = np.array([item for sublist in zip(z[:,0:2],x_y[:,0:2]) for item in sublist])
  z = z[:,0]
  q = np.ones_like(z)
  a3 = np.array([item for sublist in zip(q,z) for item in sublist])
  a4 = np.array([item for sublist in zip(z,q) for item in sublist])
  A = np.column_stack((a1,a2,a3,a4))

  return np.linalg.lstsq(A, x_y_prime, rcond=None)

In [ ]:
# Function to transform the (x,y) using (S, t)
def transform_points(file, kernel):
  """ 
  Transform (x,y) to (x',y') using (S, t)

  input: points (nd.array)
         matrix (3 x 3)

  return: warped points
  """

  # Convert points to np.array
  points = np.array(file)

  # Pad the data with ones, so that our transformation can do translations
  pts_pad = np.hstack([points, np.ones((points.shape[0], 1))])
  
  # Compute dot product of kernel and points
  points_warp = np.dot(pts_pad, kernel.T)

  return points_warp[:, :-1]